In [1]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import MinMaxScaler
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.metrics import r2_score
from statsmodels.tsa.stattools import adfuller
from sklearn.impute import KNNImputer

import warnings
warnings.simplefilter('ignore')
pd.set_option('display.max_columns', None)

In [2]:
df = pd.read_csv("../working_data/dataset.csv", index_col=0)

In [3]:
df = df[df["Category"] != "Industrial Crops"]

## Creating working data frame

In [5]:
#df['Year'] = pd.to_datetime(df['Year'], format='%Y')
#df.set_index("Year", inplace=True)
# getting list of names
categories = df["Category"].unique()

## Creating the metrics data frame

In [6]:
def calculate_metrics(df, crop_name, y_true, y_pred):
    mae = metrics.mean_absolute_error(y_true, y_pred)
    mse = metrics.mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_true, y_pred)
    y_test_avg = pd.DataFrame(y_true).mean()
    y_pred_avg = y_pred.mean()

    df.loc[crop_name] = [mae, mse, rmse, r2, y_test_avg[0], y_pred_avg]

In [7]:
quality_metrics = ['RF_MAE', 'RF_MSE', 'RF_RMSE', "RF_R2", "Actual Yield", "Predicted Yield"]
error_df = pd.DataFrame(columns = quality_metrics, index = categories)
#display(error_df)

## Support Vector Regression

### Setting up dataframe

#### - Solving for stationarity

In [8]:
imputer = KNNImputer(n_neighbors=4)
adf_df = pd.DataFrame(columns=df.columns)

for cate in categories: 
    # Selects category
    crop_df = df[df["Category"] == cate]
    num_cols = crop_df.select_dtypes(include=[np.number])
    # Selects column for ADF Test
    for col in num_cols:
        if crop_df[col].count() < 5:
            continue
        adf_result = adfuller(crop_df[col])
        while adf_result[1] > 0.05:
            crop_df[col] = crop_df[col].diff()
            crop_df[num_cols.columns] = imputer.fit_transform(crop_df[num_cols.columns])
            adf_result = adfuller(crop_df[col])

    adf_df = pd.concat([adf_df, crop_df])

#### - Transforming time series data to supervised learning (adding lags to variables)

In [9]:
main_df = pd.DataFrame(columns=adf_df.columns)
imputer = KNNImputer(n_neighbors=4)

# Sliding Window
for cate in categories: 
    # Selects category
    crop_df = adf_df[adf_df["Category"] == cate]
    num_cols = crop_df.select_dtypes(include=[np.number])
    for col in num_cols.columns:   
        for i in range(1, 3):
            crop_df[f'{col}-{i}'] = crop_df[col].shift(i)

    main_df = pd.concat([main_df, crop_df])
    num_cols = main_df.select_dtypes(include=[np.number])
    main_df[num_cols.columns] = imputer.fit_transform(main_df[num_cols.columns])

## Running the model

In [14]:
scaler = MinMaxScaler()
for cat in categories:
   crop_df = main_df[main_df['Category'] == cat]
   encoded_df = pd.get_dummies(crop_df[["Season"]], prefix='', prefix_sep='')
   encoded_df = encoded_df.drop(encoded_df.columns[-1], axis=1)
   
   y = crop_df[['Yield (tonnes/ha)']]
   x = crop_df.drop(['Yield (tonnes/ha)', "Item", "Category", "Season"], axis=1)
   x = pd.concat([x, encoded_df], axis=1)
   

   mean = x.mean()
   std = x.std()
   lower_bound = mean - 2 * std
   upper_bound = mean + 2 * std
   x = x.clip(lower_bound, upper_bound, axis=1)

   x = scaler.fit_transform(x)

   x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, shuffle=False)
   regressor = SVR(kernel="rbf", C=100, epsilon=0.15)
   regressor.fit(x_train, y_train)

   #prediction of new value
   y_pred = regressor.predict(x_test)
   calculate_metrics(error_df, cat, y_test, y_pred)

## Result

In [15]:
error_df

,RF_MAE,RF_MSE,RF_RMSE,RF_R2,Actual Yield,Predicted Yield
Vegetables and Legumes,27.484351,2650.544616,51.48344,0.203412,45.004377,29.583396
Cereals,1.063254,1.901629,1.378996,0.702423,0.022847,-0.156352
Fruits and Nuts,4.778909,38.301243,6.1888,0.297543,9.75563,10.762902
